# Introduction

### Data Cleaning Project Using Audible Dataset

The data that will be used in this project was collected via web scraping from Audible India's website. The dataset was uploaded on Kaggle and contains relevant information on audiobooks from 1998 to 2025 (pre-planned releases).

In [2]:
# import libraries
import pandas as pd
import zipfile
import re
import kaggle

The Kaggle API was used to download the dataset to my computer

In [1]:
# download dataset
!kaggle datasets download -d snehangsude/audible-dataset

In [3]:
# extract csv from zip file
zipfile_name = 'audible-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [92]:
# read in csv file
audible = pd.read_csv('audible_uncleaned.csv')

### First Glance at the Data

In [85]:
audible.describe()

,name,author,narrator,time,releasedate,language,stars,price
count,87489,87489,87489,87489,87489,87489,87489,87489
unique,82767,48374,29717,2284,5058,36,665,1011
top,The Art of War,"Writtenby:矢島雅弘,石橋遊",Narratedby:anonymous,2 mins,16-05-18,English,Not rated yet,586.00
freq,20,874,1034,372,773,61884,72417,5533


In [86]:
audible.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [87]:
audible.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


About this dataset:

- Size: There are 87489 rows of data, which means there are 87489 books in this dataset (given there are no duplicates)

- Columns: There are 8 columns that represent the relevant metadata about each book

- Data Types: Every column is an "object" datatype, some columns may need to be changed

- No Missing Data: All columns match the total number of rows in the dataset which means we do not have to handle missing values

### Data Cleaning

#### Standardize author column

The author column was cleaned by removing the 'Writtenby:' at the start of each value

In [93]:
# Clean author names by removing repeating string
audible['author'] = audible['author'].str.replace('Writtenby:', '')

#### Standardize narrator column

The narrator column was cleaned similar to the author column

In [94]:
# Clean narrator names by removing repeating string
audible['narrator'] = audible['narrator'].str.replace('Narratedby:', '')

#### Convert to DateTime type
The release date column was converted to a date time column to reflect US dates instead of the Indian date format

In [95]:
# Convert release date to date time type
audible['releasedate'] = pd.to_datetime(audible['releasedate'])

#### Convert time column to minutes

The time column contains a string with the duration given in hours and minutes. To get all rows into a uniform format, the following steps were taken to convert all rows into minutes:

In [96]:
# Convert duration string into minutes
for i in range(audible.time.shape[0]):
    total = 0
    # find the numeric value for hour and minute 
    time_parts = re.findall(r'(\d+) hr?|(\d+) min?', audible.loc[i, 'time'])
    for part in time_parts:
        if part[0]:  # hours part
            total += int(part[0]) * 60
        if part[1]:  # minutes part
            total += int(part[1])
    audible.loc[i, 'time'] = total
    
# Convert time to numeric data type

audible['time'] = pd.to_numeric(audible['time'])

#### Separate star ratings from number of ratings

The stars column in the df contained a star rating out of 5 as well as the number of ratings each book recieved. These two values were separated into separate columns

In [97]:
# Create new column to separate number of ratings from star ratings
audible['ratings'] = audible['stars']

In [98]:
# Extract number from ratings and store in ratings column
s = audible['ratings']

audible['ratings'] = s.str.extract(pat='(\d+) ratings')

# Any rows that had no matches were given a 0
audible['ratings'] = audible['ratings'].fillna(0)

# Convert column to numeric data type
audible['ratings'] = pd.to_numeric(audible['ratings'])

In [99]:
# Extract star rating and store in stars column
t = audible['stars']

audible['stars'] = t.str.extract(pat='(\d*\.?\d*) out of 5 stars')

# Any rows that did not have a match were given a 0
audible['stars'] = audible['stars'].fillna(0)

# Convert column to numeric data type
audible['stars'] = pd.to_numeric(audible['stars'])

#### Standardize language column

After taking a look at the unique values, there appeared to be differing capitlizations.

In [100]:
audible.language.unique()

array(['English', 'Hindi', 'spanish', 'german', 'french', 'catalan',
       'swedish', 'italian', 'danish', 'finnish', 'dutch', 'hebrew',
       'russian', 'polish', 'galician', 'afrikaans', 'icelandic',
       'romanian', 'japanese', 'tamil', 'portuguese', 'urdu', 'hungarian',
       'czech', 'bulgarian', 'mandarin_chinese', 'basque', 'korean',
       'arabic', 'greek', 'turkish', 'ukrainian', 'slovene', 'norwegian',
       'telugu', 'lithuanian'], dtype=object)

In [101]:
audible.language = audible.language.str.capitalize()

#### Convert price column to numeric data type

Price column was given a numeric data type for future analyses

Upon an attempt to convert the price column to a numeric data type, it was discovered that not all rows contained a numeric value:

In [102]:
audible['price'] = audible['price'].str.replace(',', '').astype(float)

ValueError: could not convert string to float: 'Free'

There are 338 values where the price = "Free"

In [103]:
audible['price'].loc[audible['price'] == 'Free']

983      Free
1414     Free
1513     Free
2932     Free
4547     Free
         ... 
80021    Free
81198    Free
82644    Free
86771    Free
86927    Free
Name: price, Length: 338, dtype: object

All values where price = 'Free' was converted to 0

In [104]:
audible['price'] = audible['price'].str.replace(',', '')

audible.loc[audible.price == 'Free', 'price'] = 0

audible['price'] = pd.to_numeric(audible['price'])

We can see that the number of values is the same as previously: 338

The values were successfully converted

In [108]:
audible.loc[audible.price == 0, 'price']

983      0.0
1414     0.0
1513     0.0
2932     0.0
4547     0.0
        ... 
80021    0.0
81198    0.0
82644    0.0
86771    0.0
86927    0.0
Name: price, Length: 338, dtype: float64

## Final Dataset

We now have the final cleaned dataset ready for further exploration.

Some potential questions to look into:

1. Top languages across audiobooks

2. Average duration

3. Popular narrators

4. Popular authors

In [109]:
audible.head()

,name,author,narrator,time,releasedate,language,stars,price,ratings
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,140,2008-04-08,English,5.0,468.0,34
1,The Burning Maze,RickRiordan,RobbieDaymond,788,2018-01-05,English,4.5,820.0,41
2,The Deep End,JeffKinney,DanRussell,123,2020-06-11,English,4.5,410.0,38
3,Daughter of the Deep,RickRiordan,SoneelaNankani,676,2021-05-10,English,4.5,615.0,12
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,600,2010-01-13,English,4.5,820.0,181


In [110]:
audible.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         87489 non-null  object        
 1   author       87489 non-null  object        
 2   narrator     87489 non-null  object        
 3   time         87489 non-null  int64         
 4   releasedate  87489 non-null  datetime64[ns]
 5   language     87489 non-null  object        
 6   stars        87489 non-null  float64       
 7   price        87489 non-null  float64       
 8   ratings      87489 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 6.0+ MB


In [111]:
audible.to_csv('audible_cleaned.csv', index=False)